Доработанный скрипт site_availability.

Проверяет доступность по списку сайтов, выгружает статус, заголовок, а также адреса электронной почты на главной странице и странице контактов.

Последние правки скрипта: 2022-01-15

Для данной задачи Selenium — плохое решение, если оперативной памяти немного, он виснет, приходится перезагружать браузер. Для корректной работы лучше перезагружать через каждые 500 ссылок.

Вероятно, для аналогичной задачи упрошу код и сделаю все на реквесте и bs.

## Библиотеки и настройки

In [ ]:
#Загружаю нужные библиотеки

import pandas as pd
import math
import os

# регулярки
import re

# для юзер-агента
from fake_useragent import UserAgent

# для работы с буфером обмена
import pyperclip

# прогресс-бар
from tqdm.notebook import trange, tqdm

# для пауз при парсинге
from time import sleep

# импорт requests
import requests

# селениум
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service
from selenium import webdriver

from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
# настройки юзер-агента

ua = UserAgent()

# юзер-агент для Хрома
user_agent = ua.chrome

In [ ]:
test_url = 'https://intoli.com/blog/not-possible-to-block-chrome-headless/chrome-headless-test.html'

In [ ]:
# путь к файлам

file_path = 'C:/00_Data/' # путь к папке с данными
proxy_list_path = 'C:/work/keys/proxy_list.txt' # путь и названия файла со списком прокси

In [ ]:
# загружаю список прокси
# список простой текстовый файл, который содержит прокси в формате: адрес:порт логин:пароль

proxy_list = []

with open(proxy_list_path, 'r') as f:
    for line in f:
        proxy_list += [line.split()]
        
print('Рабочих прокси — ' + str(len(proxy_list)))

In [ ]:
# номер прокси

print('Номер прокси, от 0 до ' + str(len(proxy_list) - 1))
proxy_number = int(input())

In [ ]:
# настройки веб-драйвера, чтобы скрыть автоматическое управление

opt = webdriver.ChromeOptions()
opt.add_argument(f'user-agent={user_agent}')
opt.add_argument('--disable-blink-features')
opt.add_argument('--disable-blink-features=AutomationControlled')
#opt.add_argument(f'--proxy-server={proxy_list[proxy_number][0]}') # "девственный" прокси(:
opt.add_experimental_option("excludeSwitches", ["enable-automation"]) # отключает строку про удаленное управление
opt.add_experimental_option('useAutomationExtension', False)

In [ ]:
# пока не придумал нормального способа авторизации

pyperclip.copy(proxy_list[proxy_number][2])

print('Login: ' + proxy_list[proxy_number][1])
print('Pass: Пароль в буфере обмена')

In [ ]:
# запускаю драйвер, проверяю настройки

driver = webdriver.Chrome(options=opt)
driver.get(test_url)
driver.maximize_window() # раскрываю окно на максимум

## Собираю данные

### Загружаю список ссылок

In [ ]:
file_path

In [ ]:
# путь к нужным папкам в основной папке

file_path_add = file_path + input()

In [ ]:
# вытаскиваю названия файлов в список file_list

file_list = []

for root, dirs, files in os.walk(file_path_add):
    
    for filename in files:
        file_list += [filename]

In [ ]:
file_list

In [ ]:
# вариант для загрузки ссылок из одного файла

# имя файла
#need_file = input()

# выгружаю файл в фрейм
#work_links = pd.read_excel(file_path_add + need_file)

# создаю список ссылок
#url_list = list(work_links[work_links['web_site'] != 'none']['web_site'].unique())

In [ ]:
url_list = []

for file in file_list:
    work_links = pd.read_excel(file_path_add + file)
    
    url_list += list(work_links[work_links['web_site'] != 'none']['web_site'].unique())

In [ ]:
len(url_list)

### Код парсера

In [ ]:
# регулярка для поиска email

regex_mails = r'[A-Za-zА-Яа-я-_.]+@[A-Za-zА-Яа-я-_.]+\.[A-Za-zА-Яа-я]+'

Наверное, Селениум здесь лишний, и все можно сделать реквестом.

In [ ]:
# проверяю ссылки на страницы

data = [] # список для данных


for url in tqdm(url_list[6200:]): # цикл для ссылок

    mail_list = [] # временный список для почтовых адресов
        
    try:
        r = requests.get(url, headers={'User-Agent': user_agent})
        status = r.status_code
        
        if status == 200:
            driver.get(url)
            sleep(2)

            availability = 1
            title = driver.title
            mail_list += re.findall(regex_mails, driver.page_source)


            try:
                driver.find_element(By.LINK_TEXT, 'Контакты').click()
                mail_list += re.findall(regex_mails, driver.page_source)

            except:
                pass
            
        
    except:
        availability = 0
        title = 'none'        
        status = 'none'

    mail_list = list(set(mail_list))
    
    data.append([url, availability, status, title, mail_list])

In [ ]:
# Записываю в датафрейм

head_list = ['web_site', 'availability', 'status', 'title', 'mail_list']

work = pd.DataFrame(data, columns=head_list)

In [ ]:
work.sample(3)

In [ ]:
# записываю  в файл, в основную папку

work.to_excel(file_path + 'stom/site_list_13.xlsx', encoding='utf8', index=False)

In [ ]:
driver.quit()